**IMPORTAMOS LIBRERIAS**

In [1]:
import tensorflow as tf

# Set de datos MNIST 

from tensorflow.examples.tutorials.mnist import input_data

 **Zona de lectura de nuestra carpeta de imagenes** 

In [2]:
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True) 

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# **Zona de construccion de nuestra red neuronal**

En esta seccion vamos a declarar todos los elementos que van a conformar nuestra red

# La variable X sera nuestra matriz de entrada 

Note que el tamaño de nuestra matriz sera  de  `[None,784]` donde `None` sera el numero de imagenes que inicialmente no sabemos cuantas son , pero con esta palabra de alguna manera le decimos a nuestra variable de entrada que entre por bloques y por otro lado el segundo argumento sera `784` que vendrian siendo los pixeles de nuestras imagenes que para este caso seran `28x28=784`.
 

In [3]:
x=tf.placeholder(tf.float32,[None,784]) #Imagen del numero descompuesta a un vector


## La variable P sera nuestros pesos o `W`

Note que va a ser un tipo de dato variable y la vamos a inicializar con `ceros` y va a ser una matriz de 784 renglones y 10 columnas `(labels=0,1,2,...,8,9,10)` , estamos conenctando nuestra primera capa que tiene 784 pixeles con nuestra segunda capa que tiene 10 elementos en un vector .

In [4]:
P=tf.Variable(tf.zeros([784,10])) #Matriz de pesos, 784 para recibir la imagen, 10 por las posible salidas


# La variable **b** sera nuestro *`BIAS`*

Es el sesgo de nuestra red neuronal.

In [5]:
b=tf.Variable(tf.zeros([10])) #Vector con BIAS

# La variable **y** sera nuestro *`capa de salida`*

**`Recordar`** Que cuando estamos trabajando con redes neuronales estas se encuentran conformadas por nuestra capa de entrada, capa oculta y capa de salida.

Por otro lado nuestra operacion matematica sera la multiplicacion del vector de entrada por nuestra matriz de pesos :

**`y=matmul(x+w)+b`**


In [6]:
y=tf.matmul(x,P)+b #La operacion que se hara en los nodos que reciben entradas

# Matriz con las etiquetas reales

Esta matriz va a tener todas las etiquetas  `respuestas correctas` de nuestro set de datos y a su vez la vamos a usar para entrenar nnuestra red neuronal.

In [7]:
yR=tf.placeholder(tf.float32,[None,10]) # Matriz con las etiquetas REALES del set de datos

# Definir la función de costo entropia cruzada (Cross Entropy) para poder medir el error. La salida será con Softmax

* **`Softmax`** lo que va a hacer es convertir nuestro vector de 10 elementos y nos lo va a mostrar en probabilidades , es decir el numero que estemos ingresando a nuestra red tendra la probabilidad mas alta . 

* La funcion con la que vamos a estar midiendo el error en cada una de nuestras predicciones va a ser  `cross_entropy_with_logits` y le vamos a decir que `(labels=yR,logits=y)` nuestra red neuronal genero `y` comparalo con `yR` y dime que tan bien o que tan mal esta mi algoritmo para que pueda hacer los ajustes correspondientes para mejorar la prediccion

* El costo es la funcion que deseamos reducir a lo mas minimo es decir que tienda a `cero`

* Por ultimo nuestro ***optimizador*** el cual va a utilizar el metodo de `gradiente descendiente` para minimizar la funcion `costo` y ver como ajustar nuestros pesos en nuestra matriz para reducirlo. 

In [8]:
softmax=tf.nn.softmax_cross_entropy_with_logits(labels=yR,logits=y)
costo=tf.reduce_mean(softmax)
optimizador=tf.train.GradientDescentOptimizer(0.5).minimize(costo)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# Correr la gráfica computacional

1. **`Prediccion`** En esta parte lo que estamos haciendo es que de todas las probabilidades que nos dio **`softmax`** nos va a traer el numero mas alto y nos lo va a comparar el vector de salidas reales con la que estamos calculando.

2. **`accuracy`** Es para medir nuestro porcentaje de certeza .


In [9]:
#Nos da arreglo de booleanos para decirnos cuales estan bien y cuales no
prediccion = tf.equal(tf.argmax(y, 1), tf.argmax(yR, 1)) 
#Nos da el porcentaje sobre el arreglo de prediccion
accuracy = tf.reduce_mean(tf.cast(prediccion, tf.float32))
Produccion = tf.argmax(y,1)
init=tf.global_variables_initializer() # Inicializamos variables

# Entrenando el  algoritmo

Esta funcion que vamos a usar es  para ver que tan bien va a aprendiendo nuestro modelo , **`Recordar`** que nuestras variables **`placeholder`** debemos alimentarlas con `feed_dict`

In [10]:
def avance(epoca_i, sess, last_features, last_labels):
    costoActual = sess.run(costo,feed_dict={x: last_features, yR: last_labels})
    Certeza = sess.run(accuracy,feed_dict={x:mnist.validation.images,yR: mnist.validation.labels})
    print('Epoca: {:<4} - Costo: {:<8.3} Certeza: {:<5.3}'.format(epoca_i,costoActual,Certeza))

# Seccion de codigo de mas importancia dentro nuestro modelo

1. Creamos nuestra sesion para correr nuestro modelo
2. Vamos a usar 1000 epocas 
3. Le vamos a decir a nuestro modelo que nos evalue por lotes debido a que si usamos todas nuestras imagenes no requeriria un alto costo computacional entonces usamos `next_batch(100)` en donde `lote x` sera nuestras imagenes y `lotey ` seran nuestras etiquetas de estas imagenes y le vamos a decir que nuestros lotes queremos que sean de 10 elementos .
4. Vamos a correr nuestro optimizador y lo alimentamos con nuestros lotes que van a estar encadenados con todos los procedimientos anteriores.
5. Vamos a decirle a nuestro algoritmo que cada vez que nuestra `epoca==50` nos llame a la funcion avance para medir nuestra certeza.
6. `Zona de impresiones` tener en cuenta que en esta zona vamos a evaluar son nuestros datos de testing.

In [11]:
with tf.Session() as sess:
    sess.run(init)
    for epoca_i in range(1000):
        lotex, lotey = mnist.train.next_batch(200)
        sess.run(optimizador, feed_dict={x: lotex, yR: lotey})
        if (epoca_i%50==0):
            avance(epoca_i, sess, lotex, lotey)
    print('RESULTADO FINAL: ',sess.run(accuracy, feed_dict={x: mnist.test.images,yR: mnist.test.labels}))
    print ('Resultado de una imagen',sess.run(Produccion,feed_dict={x: mnist.test.images[5].reshape(1,784)}))

Epoca: 0    - Costo: 1.76     Certeza: 0.36 
Epoca: 50   - Costo: 0.423    Certeza: 0.88 
Epoca: 100  - Costo: 0.372    Certeza: 0.902
Epoca: 150  - Costo: 0.338    Certeza: 0.905
Epoca: 200  - Costo: 0.402    Certeza: 0.905
Epoca: 250  - Costo: 0.308    Certeza: 0.91 
Epoca: 300  - Costo: 0.332    Certeza: 0.906
Epoca: 350  - Costo: 0.387    Certeza: 0.916
Epoca: 400  - Costo: 0.31     Certeza: 0.917
Epoca: 450  - Costo: 0.27     Certeza: 0.917
Epoca: 500  - Costo: 0.293    Certeza: 0.921
Epoca: 550  - Costo: 0.275    Certeza: 0.92 
Epoca: 600  - Costo: 0.242    Certeza: 0.92 
Epoca: 650  - Costo: 0.214    Certeza: 0.922
Epoca: 700  - Costo: 0.385    Certeza: 0.921
Epoca: 750  - Costo: 0.284    Certeza: 0.92 
Epoca: 800  - Costo: 0.345    Certeza: 0.922
Epoca: 850  - Costo: 0.241    Certeza: 0.923
Epoca: 900  - Costo: 0.389    Certeza: 0.923
Epoca: 950  - Costo: 0.265    Certeza: 0.923
RESULTADO FINAL:  0.9218
Resultado de una imagen [1]


In [12]:
mnist.test.labels[5]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

**FINAL**